<a href="https://colab.research.google.com/github/siddadel/kalidas/blob/main/Dialogue_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tue Nov 30 20:40:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/

In [27]:
!pip install transformers --quiet
!git clone https://github.com/allenai/comet-atomic-2020
!pip install -r ./comet-atomic-2020/requirements.txt --quiet
!wget https://storage.googleapis.com/ai2-mosaic-public/projects/mosaic-kgs/comet-atomic_2020_BART.zip
!unzip comet-atomic_2020_BART.zip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
fatal: destination path 'comet-atomic-2020' already exists and is not an empty directory.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set t

In [28]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
data_root = "/content/drive/MyDrive/ANLP21/scripts_txt"
#data_root = "/content/drive/MyDrive/ANLP21/scripts_sample"
output_dir= "/content/drive/MyDrive/ANLP21/exp"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import pandas as pd
import random
import os
import re
import spacy
from collections import Counter
from joblib import Parallel, delayed
import pandas as pd
nlp = spacy.load("en_core_web_sm")

In [30]:
import sys
sys.path.insert(1,r'./comet-atomic-2020/models/comet_atomic2020_bart')
from generation_example import Comet
print("model loading ...")
comet = Comet("comet-atomic_2020_BART")
comet.model.zero_grad()
print("model loaded")

model loading ...
model loaded


In [31]:
relations = ["CapableOf",  "Desires",  "MotivatedByGoal",  "xAttr", "xNeed", "xReact", "xReason", "xWant"]

vocabulary = set()

film_bags = {}

def register_in_bag(film, token, relation):
    if film not in film_bags:
      film_bags[film] = {}
    if(relation not in film_bags[film]):
      film_bags[film][relation] = set()
    film_bags[film][relation].add(token)
    vocabulary.add(token)
    
def process_results(film, results, relations):
    for i in range(len(relations)):
      relation = relations[i]
      for result in results[i]:
        tokens = result.split(' ')
        for token in tokens:
          if token not in nlp.Defaults.stop_words:
            register_in_bag(film, token, relation)

log = []
def infer(film, item, relations):
    queries= ["{} {} [GEN]".format(item, relation) for relation in relations]
    results = comet.generate(queries, decode_method="beam", num_generate=5)
    if film not in log:
      print(film, end=",")
      log.append(film)
    process_results(film, results, relations)

In [32]:
df1 = pd.read_csv("/content/drive/MyDrive/ANLP21/data.csv")

In [33]:
# df1.apply(lambda row: infer(row['film'], row['item'], relations), axis=1)

In [ ]:
import json

def infer(item):
  queries= ["{} {} [GEN]".format(item, relation) for relation in relations]
  results = comet.generate(queries, decode_method="beam", num_generate=5)
  return results

film_group = df1.groupby('film')
for film, group in film_group:
  print(film)
  #The following three ways were quicker ways but they don't work for some reason
  # if you want to work with these make sure to convert the results array in infer into json or string before returning
  # group['inference'] = comet.generate(["{} {} [GEN]".format(group['item'], relation) for relation in relations], decode_method="beam", num_generate=5)
  # group['inference'] = group['item'].apply((lambda x: infer(x)), axis=1)
  # group['inference'] = infer(group['item'])
  # for i in range(len(relations)):
  #   # group[str(relations[i])] = group['inference'].apply((lambda x: parse_inf(x, i)), axis=1)
  #   group[str(relations[i])] = parse_inf(group['inference'], i)
  
  for i in range(len(relations)):
    group[relations[i]] = ["" for i in range(len(group))]

  rel_inf = []
  for ind in group.index:
    item = group['item'][ind]
    results = infer(item)
    for i in range(len(relations)):
      group[relations[i]][ind] = results[i]

  group.to_csv(film+'.csv')
  files.download(film+'.csv')
  break


10_things_i_hate_about_you.txt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
N